In [1]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

def translate_m2m100(english_text, src_lang="en", tgt_lang="tr"):
    tokenizer.src_lang = src_lang
    inputs = tokenizer(english_text, return_tensors="pt", padding=True)
    generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))
    return tokenizer.decode(generated_tokens[0], skip_special_tokens=True)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [2]:
english_text = "okay"
turkish_translation = translate_m2m100(english_text)
print(f"English: {english_text}")
print(f"Turkish: {turkish_translation}")

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

English: okay
Turkish: Tamam


In [4]:
def translate_file(input_file, output_file, src_lang="en", tgt_lang="tr"):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()  # Read file line by line to preserve formatting

    translated_lines = []
    for line in lines:
        if line.strip():  # Translate only non-empty lines
            translated_line = translate_m2m100(line.strip(), src_lang, tgt_lang)
            translated_lines.append(translated_line)
        else:
            translated_lines.append("")  # Preserve blank lines

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("\n".join(translated_lines))  # Join lines back with newline characters

    print(f"Translation saved to {output_file}")

translate_file("/content/sample_data/input.txt", "translated_output.txt", "en", "tr")

Translation saved to translated_output.txt


In [5]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [6]:
!pip install pdfplumber



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.2 MB/s eta 0:00:00


In [7]:
import pdfplumber

def extract_text_from_pdf(pdf_path, output_txt):
    """Extracts text from a PDF and saves it to an intermediate text file while preserving formatting."""
    text = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            extracted_text = page.extract_text()
            if extracted_text:
                # Preserve page breaks for clarity
                text += f"\n--- Page {page_num} ---\n{extracted_text}\n"
            else:
                text += f"\n--- Page {page_num} (No text extracted) ---\n"

    if not text.strip():
        print("No extractable text found in the PDF.")
        return None

    # Save extracted text to a file
    with open(output_txt, "w", encoding="utf-8") as f:
        f.write(text)

    print(f"Extracted text saved to {output_txt}")
    return output_txt

# Example usage
pdf_path = "/content/sample_data/Customer_sentiment_analysis_synopsis.pdf"
output_txt = "extracted_text.txt"
extract_text_from_pdf(pdf_path, output_txt)


Extracted text saved to extracted_text.txt


'extracted_text.txt'

In [8]:
with open("extracted_text.txt", 'r', encoding="utf-8") as f:
    extracted_text = f.read()
    print(extracted_text)


--- Page 1 ---
Guru Gobind Singh Indraprastha University
CUSTOMER SENTIMENT AND FEEDBACK ANALYSIS
MAJOR PROJECT SYNOPSIS
Submitted by: Submitted to:
Name: MS. Kanika
Gurleen Kaur Bali
Roll Number:
03290302022
Course and section: BCA M1

--- Page 2 ---
Gurleen Kaur Bali 03290302022 Major Project Synopsis
SYNOPSIS OF THE PROJECT:
Title of the Project:
Customer Sentiment and feedback Analysis
Statement about the Problem:
Customer feedback plays a crucial role in shaping business strategies. With the rise of e-commerce and
online platforms, customers frequently leave reviews about products and services. However, manually
analysing thousands of reviews is time-consuming and inefficient. Businesses need an automated
system to analyse customer sentiments accurately and provide actionable insights. This project aims to
develop a sentiment analysis system that classifies customer reviews as positive, negative, or neutral
and predicts a corresponding star rating out of 5.
Why is the Particular 

In [9]:
translate_file("extracted_text.txt", "translated_output_pdf.txt", "en", "tr")


Translation saved to translated_output_pdf.txt


In [10]:
from transformers import BertForMaskedLM, BertTokenizer
import torch
import re

# Load Turkish BERT model and tokenizer
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

def preprocess_text(file_path):
    """
    Reads and cleans the translated Turkish text from a file.
    """
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()

    # Remove unwanted characters and normalize spaces
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^\wşçöğüİıŞÇÖĞÜa-zA-Z.,!? ]", "", text)
    text = text.strip()

    return text

def compute_perplexity(text):
    """
    Computes a score for the text using the BERT Masked Language Model.
    This is used as a proxy to assess the fluency of the text.
    """
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss)

    return perplexity.item()

# Example Usage:
file_path = "translated_output_pdf.txt"
cleaned_text = preprocess_text(file_path)
perplexity_score = compute_perplexity(cleaned_text)

print(f"Perplexity Score: {perplexity_score}")


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Perplexity Score: 1.4937987327575684


In [11]:
pip install textstat


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.5 MB/s eta 0:00:00


In [15]:
import textstat

def readability_score(text):
    score = textstat.flesch_reading_ease(text)
    return score

def check_readability_of_translated_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    score = readability_score(text)
    return score

# Call this function with your file
readability_Score=check_readability_of_translated_file('translated_output_pdf.txt')
print(f"Readability Score:{readability_Score}")


Readability Score:40.35


In [16]:
from sentence_transformers import SentenceTransformer, util

def check_semantic_similarity(original, translated):
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # Pre-trained multilingual model
    embeddings = model.encode([original, translated])
    similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1])
    return similarity.item()

def check_semantic_similarity_of_translated_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        translated_text = file.read()

    # Example original sentence in English
    original = "This is a sample English sentence."

    similarity_score = check_semantic_similarity(original, translated_text)
    return similarity_score

# Call this function with your file
similarity_score=check_semantic_similarity_of_translated_file('translated_output_pdf.txt')
print(f"Semantic Similarity Score: {similarity_score}")

Semantic Similarity Score: 0.10829651355743408


In [17]:
def rate_translation(perplexity, readability, semantic_similarity):
    # Normalize Scores
    P_norm = max(0, 1 - (perplexity / 5))  # Assuming 5 as an upper limit for perplexity
    R_norm = readability / 100  # Readability score is already out of 100
    S_norm = semantic_similarity  # Already between 0-1

    # Weighted Rating Calculation
    rating = (0.4 * P_norm) + (0.3 * R_norm) + (0.3 * S_norm)

    # Scale to 10
    final_score = round(rating * 10, 2)

    return final_score


rating = rate_translation(perplexity_score,readability_Score, similarity_score)
print(f"Translation Quality Score: {rating}/10")


Translation Quality Score: 4.34/10
